**Problem set 1.2 IO: Production Function Estimation OP and ACF**

Author: Jordi Torres 

Start Date: 11/12/2025



In [1]:
using StatFiles, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/GMdata.dta" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = DataFrame(load(data_path))



Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537
6,2,366,88,5.19584,0.518794,3.46187,3.99812,2.82213,2.59
7,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376
8,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182
9,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101


In [2]:
##1.Ols on the full panel

df= dropmissing(main, [:ldsal, :lemp, :ldnpt]) ##no missings


X= hcat(
    ones(length(df.ldsal)),
    df.ldnpt,
    df.lemp
)

beta_ols= (X'X)\(X'df.ldsal)


3-element Vector{Float64}:
 3.0798222249143237
 0.42719629387166963
 0.5433117916971737

In [3]:
using FixedEffectModels


reg_ols1 = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)

                             FixedEffectModel                             
Number of obs:                  2971  Converged:                      true
dof (model):                       2  dof (residuals):                1398
R²:                            0.919  R² adjusted:                   0.919
F-statistic:                 7749.51  P-value:                       0.000
             Estimate  Std. Error   t-stat  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
ldnpt        0.427196   0.0190257  22.4537    <1e-94   0.389874   0.464518
lemp         0.543312   0.0241517  22.4958    <1e-95   0.495934   0.590689
(Intercept)  3.07982    0.0610165  50.4753    <1e-99   2.96013    3.19952


In [4]:
#we define a dataset where only the firms appearing in all years stay. Balanced.

using DataFrames


df1 = dropmissing(df, :yr)
T   = length(unique(df1.yr))

good = combine(groupby(df1, :index)) do sdf
    balanced=
    all(.!ismissing.(sdf.ldsal)) &&
    length(unique(sdf.yr)) == T
    (; keep = balanced)
end

filter_k= good[good.keep, :index]
df_balanced = df1[in.(df1.index, Ref(filter_k)), :]


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8,Float32,Float32,Float32,Float32?,Float32?,Float32?
1,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587
2,8,355,78,7.04852,2.91777,5.61488,4.31813,2.59354,3.82598
3,8,355,83,6.47044,1.89462,4.90529,4.5393,1.73744,2.56534
4,8,355,88,6.67208,2.02009,5.25774,4.52065,2.51504,2.5724
5,13,283,73,7.37242,3.05965,6.65504,6.0612,4.49305,4.47722
6,13,283,78,7.87958,3.29595,6.77531,6.35875,4.82749,4.83307
7,13,283,83,8.17725,3.53596,7.23209,6.61925,5.33549,5.75958
8,13,283,88,8.44212,3.65716,7.5052,7.21734,6.0851,6.22851
9,16,280,73,4.1332,-1.60944,1.99029,0.951125,-1.21886,-0.731078


In [5]:
reg_ols2 = reg(
    df_balanced,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)

                             FixedEffectModel                             
Number of obs:                   856   Converged:                      true
dof (model):                       2   dof (residuals):                 212
R²:                            0.913   R² adjusted:                   0.913
F-statistic:                 1486.57   P-value:                       0.000
             Estimate  Std. Error    t-stat  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
ldnpt        0.533454   0.0355608  15.0012     <1e-34   0.463356   0.603552
lemp         0.428931   0.0483986   8.86247    <1e-15   0.333527   0.524336
(Intercept)  2.72346    0.125047   21.7794     <1e-55   2.47696    2.96995


In [6]:
#first differences

using ShiftedArrays

    df3 = deepcopy(main)

    sort!(df3, [:index, :yr])

    df3 = transform(
        groupby(df, :index),
        :ldsal => (x -> x .- lag(x, 1) ) => :dif_ldsal,
        :ldnpt => (x -> x .- lag(x, 1)) => :dif_ldnpt,
        :lemp => (x -> x .- lag(x, 1)) => :dif_lemp,
    )

    first_dif= dropmissing(df3, [:dif_ldsal, :dif_ldnpt, :dif_lemp])

    reg_ols2 = reg(
    first_dif,
    @formula(dif_ldsal ~ dif_ldnpt + dif_lemp),
    Vcov.cluster(:index)
 )


                              FixedEffectModel                              
Number of obs:                   1571   Converged:                       true
dof (model):                        2   dof (residuals):                  867
R²:                             0.530   R² adjusted:                    0.529
F-statistic:                  256.209   P-value:                        0.000
              Estimate  Std. Error    t-stat  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
dif_ldnpt    0.0542293   0.0291938   1.85756    0.0636  -0.0030695   0.111528
dif_lemp     0.87041     0.0474901  18.3282     <1e-62   0.777201    0.963619
(Intercept)  0.170473    0.0126367  13.4903     <1e-37   0.145671    0.195275


In [7]:
#within group

reg_fe = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp + fe(index)), 
    Vcov.cluster(:index)
)


                          FixedEffectModel                          
Number of obs:               2440   Converged:                   true
dof (model):                    2   dof (residuals):              869
R²:                         0.976   R² adjusted:                0.963
F-statistic:              207.171   P-value:                    0.000
R² within:                  0.536   Iterations:                     1
       Estimate  Std. Error    t-stat  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────
ldnpt  0.31122    0.0368271   8.45085    <1e-15   0.23894    0.383501
lemp   0.752598   0.0606898  12.4007     <1e-31   0.633483   0.871714


In [8]:
using DataFrames, GLM, StatsBase

df.survived .= in.(df1.index, Ref(filter_k)) ##whether the firm survives the four years. 


probit = glm(@formula(survived ~  ldrnd + ldinv), df, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

survived ~ 1 + ldrnd + ldinv

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       z  Pr(>|z|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.34038     0.0492047  -27.24    <1e-99  -1.43682    -1.24394
ldrnd         0.258931    0.0249496   10.38    <1e-24   0.21003     0.307831
ldinv         0.0791476   0.0228622    3.46    0.0005   0.0343386   0.123957
────────────────────────────────────────────────────────────────────────────

**Olly Pakes**



In [9]:
##Olly/Pakes 

#step 1

Kit= df.ldnpt
Iit= df.ldinv
Lit= df.lemp
Yit= df.ldsal


#Flexible function that I can use with choosen number of polynomials K.
function poly2(k, i, l)
    n = length(k)
    X = hcat(
        ones(n),
        l,
        k, i, 
        k.^2, i.^2,
        k .* i, 
        k.^3, i.^3, 
        k.^2 .* i, 
        i.^2 .* k, 
    )
    return X
end

X= poly2(Kit, Iit, Lit)

Beta1= (X'X)\(X'Yit)

beta_l_hat= Beta1[2] #first step output 1

#I need standard errors on this...clustered at the firm level. To code! 

y_hat= X*Beta1

df.ϕ_t_hat= y_hat .- beta_l_hat .* Lit # first step output 2



2971-element Vector{Float64}:
 6.209781430835834
 6.250502801427069
 5.81039690700373
 4.095766380548965
 4.575993155774615
 4.575506937899413
 3.7173814404073697
 4.201416215664165
 4.415133564487948
 4.519976453972198
 ⋮
 3.743532192551039
 4.0060775860474145
 4.599241630536058
 4.805087902977886
 5.189322623894176
 5.524873258596808
 5.636494889143745
 5.6037225846508925
 3.939060228340184

In [10]:
using DataFrames, ShiftedArrays

sort!(df, [:index, :yr])

df2 = transform(
    groupby(df, :index),
    :yr => (x -> lead(x, 1)) => :lead_yr
)

df2.survived= .!ismissing.(df2.lead_yr)

#drop last year of the panel 

data_step2= filter(row->row.yr<88, df2)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,lead_yr
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Bool,Float64,Int8?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,true,6.20978,78
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,true,6.2505,83
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,false,5.8104,missing
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,true,4.09577,83
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,true,4.57599,88
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,true,3.71738,83
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,false,4.20142,missing
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,true,4.41513,78
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,true,4.51998,83


In [11]:
function poly3(k, i)
    n = length(k)
    X = hcat(
        ones(n),
        k, i, 
        k.^2, i.^2,
        k .* i, 
        k.^3, i.^3, 
        k.^2 .* i, 
        i.^2 .* k, 
    )
    return X
end

vars = [:survived, :ldrnd, :ldinv]

df_clean = dropmissing(data_step2, vars)

Iit= df_clean.ldinv
Kit= df_clean.ldnpt
y= df_clean.survived



X= poly3(Iit,Kit)


probit = glm(X, y, Binomial(), ProbitLink())

df_clean.P̂ = predict(probit)

df_clean_small = select(df_clean, :yr, :index, :P̂)


Row,yr,index,P̂
,Int8?,Int16?,Float64
1,73,1,0.813965
2,78,1,0.794649
3,83,1,0.510251
4,78,2,0.58963
5,83,2,0.651903
6,78,3,0.549118
7,83,3,0.587008
8,73,4,0.68706
9,78,4,0.667888


In [12]:
#step 3. 

df_clean


df_total = innerjoin(
    df,
    select(df_clean, :yr, :index, :P̂),
    on = [:index, :yr]
)

#we generate t+1 leads. 
df_total_final= transform(groupby(df_total, :index),
    :ldsal => (x->lead(x)) => :ldsal_lead,
    :lemp => (x->lead(x)) => :lemp_lead,
    :ldnpt => (x->lead(x)) => :ldnpt_lead,
    :ldrnd => (x->lead(x)) => :ldrnd_lead

)





Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Bool,Float64,Float64,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,false,6.20978,0.813965,7.09394,2.67104,7.18584,2.04228
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,false,6.2505,0.794649,6.4082,2.00647,7.4539,2.0334
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,false,5.8104,0.510251,missing,missing,missing,missing
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,false,4.09577,0.58963,4.37366,0.640801,3.4784,1.77073
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,false,4.57599,0.651903,missing,missing,missing,missing
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,false,3.71738,0.549118,3.85115,-0.478036,2.69638,0.265216
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,false,4.20142,0.587008,missing,missing,missing,missing
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,false,4.41513,0.68706,4.45412,0.330023,3.32176,0.366463
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,false,4.51998,0.667888,4.19622,0.276115,3.16302,-0.111755


In [13]:

#filter for no missing in these key variables

df_total_final=dropmissing(df_total_final, [:ldsal_lead, :ldnpt_lead, :lemp_lead, :P̂, :ϕ_t_hat , :ldnpt, :ldrnd, :ldrnd_lead])

#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
k_t=df_total_final.ldnpt

p_t=df_total_final.P̂
phi_t= df_total_final.ϕ_t_hat

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5246170338398485

In [14]:
function nonlinear_solv(β_k, p_t, y)

            ωt= phi_t .- β_k .* k_t
            y_reg= y .- β_k .* k_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 



nonlinear_solv (generic function with 1 method)

In [15]:
   using Optim
    θ0= [0.4]

   ob1(βk) = nonlinear_solv(βk, p_t, y)

   res1 = optimize(ob1, 0.01, 1.0, Brent())

   β_k = Optim.minimizer(res1)


   df_total_final.ω_t_noRD= phi_t .- β_k .* k_t 



1079-element Vector{Float64}:
 3.1185153347689782
 3.0976033215295797
 3.0402723744060753
 2.9591584564901856
 3.0815895960468653
 3.062505002270936
 3.085597922754555
 3.095893243517783
 3.020479070266043
 3.070016094817913
 ⋮
 2.9976995075699695
 2.9701174262201944
 2.94973690006776
 3.0447059147536386
 2.9087155459560305
 3.163032510431226
 2.955518791320049
 3.0585459566061006
 2.993248757062503

In [16]:
##adding rd_investment Rt+1​=(1−δR​)Rt​+ItR


##Olly/Pakes 

#step 1

Kit= df_total_final.ldnpt
Iit= df_total_final.ldinv
Lit= df_total_final.lemp
Yit= df_total_final.ldsal
RDt= df_total_final.ldrnd


#Flexible function that I can use with choosen number of polynomials K.
function poly3(k, i, l, rd)
    n = length(k)
    X = hcat(
        ones(n),
        l,
        k, i, rd,
        k.^2, i.^2, rd.^2, 
        k .* i, k .* rd, i .* rd, 
        k.^3, i.^3, rd.^3,
        k.^2 .* i, k.^2 .* rd,
        i.^2 .* k, i.^2 .* rd,
        rd.^2 .*k, rd.^2 .*i
    )
    return X
end

X= poly3(Kit, Iit, Lit, RDt)

Beta1= (X'X)\(X'Yit)

beta_l_hat_2= Beta1[2] #first step output 1

#I need standard errors on this...clustered at the firm level. To code! 

y_hat= X*Beta1

df_total_final.ϕ_t_hat_2= y_hat .- beta_l_hat_2 .* Lit # first step output 2


1079-element Vector{Float64}:
 6.279566377214945
 6.286203657101645
 4.064762566249602
 3.4335032698670878
 4.315487611271051
 4.522719834573726
 5.759887833306282
 5.839307085155932
 5.453643641600584
 5.516909535958735
 ⋮
 4.011523248979845
 4.664151791207007
 4.611514284835647
 5.407724257239273
 5.8355428720611835
 6.582161100177496
 3.448763081530395
 4.640555539717655
 5.725457112653603

In [17]:

#filter for no missing in these key variables

#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
rd_tlead= df_total_final.ldrnd_lead

k_t=df_total_final.ldnpt
rd_t=df_total_final.ldrnd

p_t=df_total_final.P̂
phi_t= df_total_final.ϕ_t_hat_2

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5246170338398485

In [18]:
function nonlinear_solv_2(β, p_t, y)
            β_k, β_rd= β

            ωt= phi_t .- β_k .* k_t - β_rd .* rd_t
            y_reg= y .- β_k .* k_tlead - β_rd.*rd_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 


nonlinear_solv_2 (generic function with 1 method)

In [19]:
   using Optim
    θ0= [0.7, 0.9]

   ob2(βk) = nonlinear_solv_2(βk, p_t, y)

   res2 = optimize(ob2, θ0, NelderMead())

   β= Optim.minimizer(res2)

   β_k_hat= β[1]
   β_rd_hat=β[2]


0.039999969741691455

**Note: this can get better coded... quite rough right now...**

**Question 2.6** 

In [20]:
#for the first case. 

df_total_final.ω_t_hat_2= ωt= phi_t .- β_k_hat .* k_t - β_rd_hat .* rd_t

df_total_final2= transform(groupby(df_total_final, :yr),
:ω_t_hat_2 => mean => :ω_t_hat_average,
:ldsal => (x-> exp.(x)./sum(exp.(x))) => :sjt,
:ldsal => (x -> length(x)) => :Nt
)

sjt= df_total_final2.sjt
Nt= df_total_final2.Nt
ω_t_hat_average= df_total_final2.ω_t_hat_average
ω_t_hat_j=df_total_final2.ω_t_hat_2

df_total_final2.covariance_op= (sjt .- (1.0 ./Nt)) .*(ω_t_hat_j .-ω_t_hat_average)


df_total_final2= transform(groupby(df_total_final2, :yr), 
:covariance_op => sum => :covariance_op_average,

)

results= [df_total_final2.ω_t_hat_average, df_total_final2.covariance_op_average]


2-element Vector{Vector{Float64}}:
 [3.1186527141349147, 3.082241518611156, 3.082241518611156, 3.082241518611156, 3.1186527141349147, 3.082241518611156, 3.1186527141349147, 3.082241518611156, 3.1186527141349147, 3.082241518611156  …  3.082241518611156, 3.082241518611156, 3.1186527141349147, 3.082241518611156, 3.082241518611156, 3.1186527141349147, 3.082241518611156, 3.082241518611156, 3.1186527141349147, 3.082241518611156]
 [0.1413255082623921, 0.1621055960648109, 0.1621055960648109, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109  …  0.1621055960648109, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109, 0.1621055960648109, 0.1413255082623921, 0.1621055960648109]

In [21]:
##for the second case. 


df_total_final2= filter(row->row.sic3==375, df_total_final)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ω_t_noRD,ϕ_t_hat_2,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Bool,Float64,Float64,Float32,Float32,Float32,Float32,Float64,Float64,Float64
1,626,375,73,5.50471,0.938835,3.32937,0.943609,-0.4809,1.75769,false,4.51111,0.643522,5.59164,1.03496,3.13264,0.388936,3.0503,4.53658,3.23178
2,626,375,78,5.59164,1.03496,3.13264,1.35218,0.388936,2.09168,false,4.44865,0.677755,5.4656,1.07432,3.96635,0.804787,3.07416,4.42376,3.1624


In [22]:
df_total_final

Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ω_t_noRD,ϕ_t_hat_2,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Bool,Float64,Float64,Float32,Float32,Float32,Float32,Float64,Float64,Float64
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,false,6.20978,0.813965,7.09394,2.67104,7.18584,2.04228,3.11852,6.27957,3.39652
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,false,6.2505,0.794649,6.4082,2.00647,7.4539,2.0334,3.0976,6.2862,3.34681
3,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,false,4.09577,0.58963,4.37366,0.640801,3.4784,1.77073,3.04027,4.06476,3.06309
4,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,false,3.71738,0.549118,3.85115,-0.478036,2.69638,0.265216,2.95916,3.4335,2.78855
5,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,false,4.41513,0.68706,4.45412,0.330023,3.32176,0.366463,3.08159,4.31549,3.0743
6,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,false,4.51998,0.667888,4.19622,0.276115,3.16302,-0.111755,3.06251,4.52272,3.18705
7,7,394,73,7.54862,3.50255,6.35166,5.47065,3.94797,4.8184,false,5.87248,0.824826,7.52507,3.35197,6.3532,3.48677,3.0856,5.75989,3.07601
8,7,394,78,7.52507,3.35197,6.3532,5.32021,3.48677,4.58983,false,5.88346,0.803268,6.91924,2.93055,6.4097,3.43927,3.09589,5.83931,3.17327
9,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587,true,5.51123,0.820489,7.04852,2.91777,5.61488,2.59354,3.02048,5.45364,3.09882


In [23]:
##Question 2.7

#ols
df.ω_t_ols= df.ldsal .- reg_ols1.coef[1] .- reg_ols1.coef[2] .* df.ldnpt .- reg_ols1.coef[3] .* df.lemp

#FE
df.ω_t_fe = df.ldsal  .- reg_fe.coef[1].* df.ldnpt  .- reg_fe.coef[2].* df.lemp



2971-element Vector{Float64}:
 2.9365356818591604
 2.847342575695577
 2.5783329693928234
 2.81700241320292
 2.8088469129493534
 3.727991116807022
 3.679699925799152
 3.3717522161465916
 3.146626078285593
 3.171946770403968
 ⋮
 3.0873424894168746
 3.017885257000613
 3.8173224678939426
 3.3440766361944156
 3.8361915779410096
 2.7398060107988447
 2.698603419586906
 3.388253404290575
 4.430773234760476

In [24]:
df_total_final

Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead,ω_t_noRD,ϕ_t_hat_2,ω_t_hat_2
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32?,Bool,Float64,Float64,Float32,Float32,Float32,Float32,Float64,Float64,Float64
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,false,6.20978,0.813965,7.09394,2.67104,7.18584,2.04228,3.11852,6.27957,3.39652
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,false,6.2505,0.794649,6.4082,2.00647,7.4539,2.0334,3.0976,6.2862,3.34681
3,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,false,4.09577,0.58963,4.37366,0.640801,3.4784,1.77073,3.04027,4.06476,3.06309
4,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,false,3.71738,0.549118,3.85115,-0.478036,2.69638,0.265216,2.95916,3.4335,2.78855
5,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,false,4.41513,0.68706,4.45412,0.330023,3.32176,0.366463,3.08159,4.31549,3.0743
6,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,false,4.51998,0.667888,4.19622,0.276115,3.16302,-0.111755,3.06251,4.52272,3.18705
7,7,394,73,7.54862,3.50255,6.35166,5.47065,3.94797,4.8184,false,5.87248,0.824826,7.52507,3.35197,6.3532,3.48677,3.0856,5.75989,3.07601
8,7,394,78,7.52507,3.35197,6.3532,5.32021,3.48677,4.58983,false,5.88346,0.803268,6.91924,2.93055,6.4097,3.43927,3.09589,5.83931,3.17327
9,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587,true,5.51123,0.820489,7.04852,2.91777,5.61488,2.59354,3.02048,5.45364,3.09882


In [25]:
#plot trends 

df_allmeasures = innerjoin(
    df,
    select(df_total_final, :yr, :index, :ω_t_noRD , :ω_t_hat_2),
    on = [:index, :yr]
)


 g = groupby(df_allmeasures, :yr)

out= combine(g, 

            :ω_t_noRD=> mean => :ω_t_1 ,
            :ω_t_hat_2=> mean => :ω_t_2 ,
            :ω_t_ols => mean => :ω_t_ols ,
            :ω_t_fe => mean => :ω_t_fe
        )



##nice results!


Row,yr,ω_t_1,ω_t_2,ω_t_ols,ω_t_fe
,Int8?,Float64,Float64,Float64,Float64
1,73,3.0564,3.11865,-0.106186,3.16725
2,78,3.06012,3.08224,-0.0599187,3.23616


**Question 2.8**

The issue is that then 

$$

 y_{it}  + p_{it}= \beta_{0} + \beta_{l}l_{it} + \beta_{l}l_{it} + \omega_{it} + \epsilon_{it}

$$

$$

 y_{it} = \beta_{0} + \beta_{l}l_{it} + \beta_{l}l_{it} + \omega_{it} -p_{it} + \epsilon_{it}

$$

The error also contains $p_{it}$ which is endogeneous and can determine labor or capital , conditional on productivity, choices. For example, firms, independent of productivity, may have prestige, market power over a region that leads to higher prices than other firms. This is not solved by OP.  

Solution: either we defend that $$ (l_{it}, k_{it}|\omega_{it}) \perp p_{it}$$ -i.e homogeneous goods industry, small markups etc- or we find an instrument that affects prices and is exogeneous to input choices (i.e demand side, for example) - other product characteristics (á la BLP)

**Question 2.9**
Now we will estimate the production function using the methodology developed by ACF. The idea is the following: 


We have the following production function:

$$y_{it}= \beta_{0} + \beta_{l} l_{it} + \beta_{k} k_{it} + \omega_{it} + \epsilon_{it} $$

Under ACF we make the following assumptions:

1. **Information set**: the firm information set at period $t$, denoted $I_{t}$, includes current and past productivity shocks $\{\omega_{\tau}\}_{\tau=0}^{t}$ but not future shocks $\{\omega_{\tau}\}_{\tau=t+1}^{\infty}$. Transitory shocks satisfy $ \mathbf{E}[\epsilon_{it}|I_{t-1}]=0$.
2. **Markov process** productivity shocks evolve according to: $$ P(\omega_{it+1}|I_{t})= P(\omega_{it+1}|\omega_{it})$$.
3. **Timing decision** Firms accumulate capital according to the process $$k_{it}=k(k_{it-1}, i_{it-1}) $$. That is, labor is the only non-dynamic input and has flexibility, while capital and investment depend on the previous period. 
4. **Scalar Unobservable** Firm's investment decisions are given by $i_{it}=f_{t}(k_{it},\omega_{it})$
5. **Invertibility** $f_{t}$ is strictly increasing in $\omega_{it}$, which implies that the function is injective and thus we can invert it: $\omega_{it}= f_{t}^{-1}(k_{t}, i_{t})$

---

Assume we observe $l_{it}, k_{it}, y_{it}, i_{it}$, amd we have T fixed and $N \rightarrow \infty$ . Then, ACF proposes to estimate the following model: 

$$ 
y_{it}= \beta_{0} + \beta_{l} l_{it} + \beta_{k} k_{it}+ f_{t}^{-1}(k_{t}, i_{t}) + \epsilon_{it}
$$

$$ 
y_{it}= \beta_{l} l_{it} + \Phi(k_{it},l_{it}, i_{it}) + \epsilon_{it}
$$

And estimate it using the condition $ \mathbf{E}[\epsilon_{it}|I_{t-1}]=0$, thus:

$$
\mathbf{E}[y_{it}-\Phi(k_{it},l_{it}, i_{it})|I_{t-1}]=0
$$
To estimate  $\hat{\Phi}$. Then in a second step we use the fact that 

$$

\mathbf{E}[\epsilon_{it+1}+ \xi_{it+1}|I_{t}]=0

$$

$$

\mathbf{E}[y_{jt+1}-\beta_{0}-\beta_{l}l_{t+1}- \beta_{k}k_{t+1} - g(\hat{P}, \hat{\Phi_{j,t}}-\beta_{0}-\beta_{l}l_{t}- \beta_{k}k_{t})|I_{t}]=0

$$

Where $I_{j,t}=1, k_{jt+1}, l_{jt}, \Phi_{jt}, l_{jt+1}$

So that the moments are: 

$$

(y_{jt+1}-\beta_{0}-\beta_{l}l_{t+1}- \beta_{k}k_{t+1} - g(\hat{P}, \hat{\Phi_{j,t}}-\beta_{0}-\beta_{l}l_{t}- \beta_{k}k_{t})) \otimes (I_{j,t} )=0

$$

We can solve this by GMM.



In [26]:
#Step 1. 

#define variables, maybe redundant, but needed. 

Kit= df_total_final.ldnpt
Iit= df_total_final.ldinv
Lit= df_total_final.lemp
Yit= df_total_final.ldsal
RDt= df_total_final.ldrnd


#Flexible function that I can use with choosen number of polynomials K.
function poly4(k, i, l)
    n = length(k)
    X = hcat(
        ones(n),

        k, i, l,
        k.^2, i.^2, l.^2,
        k .* i, k .* l, i .* l,
        k.^3, i.^3, l.^3,
        k.^2 .* i, k.^2 .* l,
        i.^2 .* k, i.^2 .* l,
        l.^2 .* k, l.^2 .* i
    )
    return X
end


X= poly4(Kit, Iit, Lit)

Beta_acf_1= (X'X)\(X'Yit)

df_total_final.phi_acf= X*Beta_acf_1

1079-element Vector{Union{Missing, Float64}}:
 7.578281993222238
 7.69736263862403
 4.019023670742969
 3.252643349066423
 4.4204161154786705
 4.678946722076802
 7.643029440318319
 7.6042003934686315
 6.9959946834005216
 7.064491758924013
 ⋮
 4.163774326529134
 5.315440238351236
 5.147891234126436
 6.265161966774641
 7.811299524468259
 8.857159392036644
 3.149430146322259
 4.215354046313531
 7.146117689612206

In [47]:
#step 2, code the gmm

#variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead


p_t=df_total_final.P̂
phi_t= df_total_final.phi_acf


#two-step GMM, step 1.

function moments(θβ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk = θβ

    ωt = phi_t .- β0 .- βl .* Lit .- βk .* Kit

    # g(ω,p) basis
    X = poly_final(ωt, p_t)

    # left-hand side for g-regression
    ytilde = y_tlead .- β0 .- βl .* l_tlead .- βk .* k_tlead

    # "profile" γ: OLS each iteration (stable)
    γ = (X'X) \ (X'ytilde)
    ghat = X * γ

    # innovation / residual
    ν = ytilde .- ghat

    # moments
    g = (Z' * ν) / n
    return g, ν, γ
end



function gmm_objective(θ, Z, n, W)
    g, _, _ = moments(θ, Z, n)
    return n * (g' * W * g)
end


function s_robust(θ, Z, n)
    _, ν, γ = moments(θ, Z, n)
    νv = vec(ν)
    M  = Z .* νv
    m=(M' * M) / n
    return m, γ
end


using ForwardDiff, LinearAlgebra

function compute_se(Z, n, θ2, W2, S1)
    gfun(θ) = moments(θ, Z, n)[1]
    D = ForwardDiff.jacobian(gfun, θ2)

    S1 = 0.5 * (S1 + S1')
    W2 = 0.5 * (W2 + W2')

    A = D' * W2 * D
    B = D' * W2 * S1 * W2 * D

    A = 0.5 * (A + A')
    B = 0.5 * (B + B')

    V = inv(A) * B * inv(A) / n
    return sqrt.(diag(V))
end



#full fledge gmm 
function gmm(Z, n, θ0)

    q = size(Z, 2)
    W1 = Matrix(I, q, q)


    #first step
    obj1(θ) = gmm_objective(θ, Z, n, W1)
    res1 = optimize(obj1, θ0, NelderMead())
    θ1 = Optim.minimizer(res1)

    #optimal weighting matrix
    S1, γ = s_robust(θ1, Z, n)
    W2 = pinv(S1)

    #second step

    obj2(θ) = gmm_objective(θ, Z, n, W2)
    res2 = optimize(obj2, θ1, NelderMead())
    θ2 = Optim.minimizer(res2)


    #standard errors

    se= compute_se(Z, n, θ2, W2, S1)



    #return

    param = hcat(
    θ2,
    se)


    return param


end 






gmm (generic function with 1 method)

In [48]:
#define instruments and inputs 

n= length(y_tlead)


#define instruments and inputs 

Z_1= hcat(
ones(length(y_tlead)), 
k_tlead,
Lit, 
phi_t
)

#first guess is OLS on this sample
X= hcat(
    ones(length(y_tlead)), 
    Lit, 
    Kit
)

θ0= (X'X)\(X'Yit)



res= gmm(Z_1, n, θ0)




3×2 Matrix{Float64}:
 7.22745   1.38743e8
 0.553176  0.0946281
 0.411941  0.0518601

**GMM all together** The idea is to pull the two moments together to exploit more information and thus be more efficient. From the steps from above we only change the first step. 




In [ ]:
function moments_stacked(θ, Z, Xφ, n)
    Kφ = size(Xφ,2)

    β0, βl, βk = θ[1:3]
    π = θ[4:3+Kφ]

    #block1
    η  = Yit .- Xφ * π
    g1 = (Xφ' * η) / n

    #block2
    φt = Xφ * π
    ωt = φt .- β0 .- βl .* Lit .- βk .* Kit

    Xg = poly_final(ωt, p_t)
    ytilde = y_tlead .- β0 .- βl .* l_tlead .- βk .* k_tlead

    # profile γ
    γ = (Xg'Xg) \ (Xg'ytilde)
    ghat = Xg * γ

    ε = ytilde .- ghat

    g2 = (Z' * ε) / n

    return vcat(g1, g2), η, ε
end



function gmm_objective_full(θ, Z, n, W, Xφ)
    g, _, _ = moments_stacked(θ, Z, Xφ, n)

    return n * (g' * W * g)
end


function s_robust_full(θ, Z, Xφ, n)
    g, η, ε = moments_stacked(θ, Z, Xφ, n)

    M1 = Xφ .* η        # n × Kφ
    M2 = Z  .* ε        # n × q2
    M  = hcat(M1, M2)   # n × q

    return (M' * M) / n
end



using ForwardDiff, LinearAlgebra

function compute_se_full(Z, Xφ, n, θ2, W2, S1)
    gfun(θ) = moments_stacked(θ, Z, Xφ, n)[1]

    D = ForwardDiff.jacobian(gfun, θ2)

    S1 = 0.5 * (S1 + S1')
    W2 = 0.5 * (W2 + W2')

    A = D' * W2 * D
    B = D' * W2 * S1 * W2 * D

    A = 0.5 * (A + A')
    B = 0.5 * (B + B')

    # solve A^{-1} B A^{-1} safely
    V = (A \ B) / A'
    V = 0.5 * (V + V') / n

    d = diag(V)
    d = map(x -> x < 0 ? NaN : x, d)
    return sqrt.(d)
end




compute_se_full (generic function with 1 method)

In [45]:
function gmm_full(Z, n, θ0, Xφ)

    Kφ = size(Xφ,2)
    q2 = size(Z,2)
    q  = Kφ + q2
    W1 = Matrix(I, q, q)

    # step 1
    obj1(θ) = gmm_objective_full(θ, Z, n, W1, Xφ)
    res1 = optimize(obj1, θ0, NelderMead())
    θ1 = Optim.minimizer(res1)

    # robust S and ridge-regularized W
    S1 = s_robust_full(θ1, Z, Xφ, n)
    S1 = 0.5*(S1 + S1')
    λ  = 1e-8 * tr(S1) / size(S1,1)
    S1r = S1 + λ*I
    W2  = inv(S1r)

    # step 2
    obj2(θ) = gmm_objective_full(θ, Z, n, W2, Xφ)
    res2 = optimize(obj2, θ1, NelderMead())
    θ2 = Optim.minimizer(res2)

    # SEs (use same S that defines W2)
    se = compute_se_full(Z, Xφ, n, θ2, W2, S1r)

    return hcat(θ2, se)
end



gmm_full (generic function with 1 method)

In [46]:
Xφ = poly4(Kit, Iit, Lit)
 Kφ = size(Xφ, 2)



n= length(y_tlead)


#define instruments and inputs 

Z_1 = hcat(ones(n), k_tlead, Lit, Kit)
  # maybe add Kit too


#first guess is OLS on this sample
X= hcat(
    ones(length(y_tlead)), 
    Lit, 
    Kit
)

θ0= (X'X)\(X'Yit)

θ0_1=vcat(θ0, Beta_acf_1)

param= gmm_full(Z_1, n, θ0_1, Xφ)
    #define theta0

22×2 Matrix{Float64}:
  3.64384    1.29037e10
  0.576638   0.072921
  0.407949   0.0392224
  2.79268    0.728205
  0.736984   0.717681
 -0.281783   0.413085
  0.376291   0.775061
 -0.196779   0.258333
 -0.534147   0.20626
  0.150374   0.278089
  ⋮          
  0.0335741  0.0349435
 -0.101589   0.034291
  0.0494365  0.0455916
 -0.180153   0.096971
  0.0437509  0.0802253
  0.260767   0.0964322
 -0.0492615  0.0814519
 -0.114423   0.10261
  0.0631131  0.0977069